# Create annual IMS cubes of Alaska site

In [14]:
import xarray as xr
import geopandas as gpd

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49344,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:49415,Total threads: 10
Dashboard: http://127.0.0.1:49424/status,Memory: 0 B
Nanny: tcp://127.0.0.1:49347,


2024-10-29 14:47:16,348 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 832, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 552, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 1956, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\core.py", line 1395, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\sno

In [57]:
year = 2023
folder = 'F:/IMS/' + str(year)
wkt = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Canada\Snow Dynamics\IMS (1 km)\ims_wkt2_2018.txt'

In [58]:
# Open IMS data as a single cube with smallest dytpe
ims = xr.open_mfdataset(folder + '/*.nc', # Downloaded as daily netCDF files
                        engine = 'netcdf4',  # Fastest
                        chunks = {'time': 1, 'x': -1, 'y': -1}, # Good chunking (purely by time-step)
                        drop_variables = 'projection').to_dataarray().squeeze().astype('uint8') # Don't need projection (will set from wkt)
ims

<xarray.DataArray (time: 365, y: 24576, x: 24576)> Size: 220GB
dask.array<astype, shape=(365, 24576, 24576), dtype=uint8, chunksize=(1, 24576, 24576), chunktype=numpy.ndarray>
Coordinates:
  * x         (x) float32 98kB -1.229e+07 -1.229e+07 ... 1.229e+07 1.229e+07
  * y         (y) float32 98kB -1.229e+07 -1.229e+07 ... 1.229e+07 1.229e+07
  * time      (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    variable  <U18 72B 'IMS_Surface_Values'
Attributes: (12/49)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    ...                         ...
    metadata_link:              https://nsidc.org/data/G02156
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...

In [59]:
# Apply custom CRS from provided WKT file
with open(wkt, 'r') as file:
    wkt = file.read()

ims.rio.write_crs(wkt, inplace = True)
ims.rio.write_nodata(0, inplace = True) # Set No Data to 0 (for later)
ims

<xarray.DataArray (time: 365, y: 24576, x: 24576)> Size: 220GB
dask.array<astype, shape=(365, 24576, 24576), dtype=uint8, chunksize=(1, 24576, 24576), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float32 98kB -1.229e+07 -1.229e+07 ... 1.229e+07 1.229e+07
  * y            (y) float32 98kB -1.229e+07 -1.229e+07 ... 1.229e+07 1.229e+07
  * time         (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    variable     <U18 72B 'IMS_Surface_Values'
    spatial_ref  int64 8B 0
Attributes: (12/50)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    ...                         ...
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...
    _FillValue:                 0

In [60]:
# Subset to bbox around Alaska site with some buffer
ims = ims[:, 11430:11530, 10160:10260]
ims

<xarray.DataArray (time: 365, y: 100, x: 100)> Size: 4MB
dask.array<getitem, shape=(365, 100, 100), dtype=uint8, chunksize=(1, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float32 400B -2.128e+06 -2.126e+06 ... -2.03e+06 -2.028e+06
  * y            (y) float32 400B -8.575e+05 -8.565e+05 ... -7.585e+05
  * time         (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    variable     <U18 72B 'IMS_Surface_Values'
    spatial_ref  int64 8B 0
Attributes: (12/50)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    ...                         ...
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...
    _FillValue:                 0

In [61]:
#test[0].rio.to_raster('test2.tif')

In [62]:
# Extent shp
area = gpd.read_file(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\Extent\Alaska_snowSites.shp", engine = 'pyogrio')
area

,Id,geometry
0,0,"POLYGON ((399535.485 7606266.209, 399535.485 7..."


In [63]:
area.crs

<Projected CRS: EPSG:32606>
Name: WGS 84 / UTM zone 6N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 150°W and 144°W, northern hemisphere between equator and 84°N, onshore and offshore. United States (USA) - Alaska (AK).
- bounds: (-150.0, 0.0, -144.0, 84.0)
Coordinate Operation:
- name: UTM zone 6N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [64]:
%%time
ims_list = [] # For input into xr.concat(), will append with months that have time-steps

# Run monthly process, will say Canada but not limited to that
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'January', 1, True) # January
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'February', 2, True) # February
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'March', 3, True) # March 
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'April', 4, True) # April 
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'May', 5, True) # May 
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'June', 6, True) # June
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'July', 7, True) # July
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'August', 8, True) # August
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'September', 9, True) # September
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'October', 10, True) # October
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'November', 11, True) # November
ims_list = snow_util.monthlyIMSprocess(ims, area, ims_list, 'December', 12, True) # December 

# Combine back into annual IMS cube
ims = xr.concat(ims_list, dim = 'time')  
ims

... January ...
Loaded into memory (shape = 31, 100, 100).
Reprojected to EPSG:32606 (shape = 31, 136, 136).
Clipped to Canada (shape = 31, 73, 47).
... February ...
Loaded into memory (shape = 28, 100, 100).
Reprojected to EPSG:32606 (shape = 28, 136, 136).
Clipped to Canada (shape = 28, 73, 47).
... March ...
Loaded into memory (shape = 31, 100, 100).
Reprojected to EPSG:32606 (shape = 31, 136, 136).
Clipped to Canada (shape = 31, 73, 47).
... April ...
Loaded into memory (shape = 30, 100, 100).
Reprojected to EPSG:32606 (shape = 30, 136, 136).
Clipped to Canada (shape = 30, 73, 47).
... May ...
Loaded into memory (shape = 31, 100, 100).
Reprojected to EPSG:32606 (shape = 31, 136, 136).
Clipped to Canada (shape = 31, 73, 47).
... June ...
Loaded into memory (shape = 30, 100, 100).
Reprojected to EPSG:32606 (shape = 30, 136, 136).
Clipped to Canada (shape = 30, 73, 47).
... July ...
Loaded into memory (shape = 31, 100, 100).
Reprojected to EPSG:32606 (shape = 31, 136, 136).
Clipped to

<xarray.DataArray (time: 365, y: 73, x: 47)> Size: 1MB
array([[[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
...
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]]], dtype=uint8)
Coordinates:
  * x            (x) float64 376B 4.002e+05 4.012e+05 ... 4.452e+05 4.462e+05
  * y            (y) float64 584B 7.679e+06 7.678e+06 ... 7.608e+06 7.607e+06
  * time         (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    variable     <U18 72B 'IMS_Surface_Values'
    spatial_ref  int64 8B 0
Attributes: (12/50)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    ...                         ...
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...
    _FillValue:                 0

In [65]:
%%time
pputil.downloadNC(ims, r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\2 IMS\ims_nc', 'IMS_' + str(year) + '.nc', type = '')

CPU times: total: 15.6 ms
Wall time: 94.4 ms
